# Stáhnutí meteodat pro stanici Praha-Ruzyně a další

In [ ]:
from datetime import datetime

import pandas as pd
from meteostat import Hourly, Stations, Daily

In [ ]:
# Najděme nejbližší stanice k Praze
lat, lon = 50.0755, 14.4378  # Praha
s = Stations()
s = s.nearby(
    lat=lat,
    lon=lon,
    radius=250000  # v m?
)
stations = s.fetch(1000)
stations   # pandas DataFrame

In [ ]:
# Použijeme české stanice s rozumnými daty (Klementinum chybí)
selected_stations = stations[(stations["hourly_start"] < pd.to_datetime("1980-01-01")) & (stations["country"] == "CZ")]
selected_stations

In [ ]:
def get_hourly_data(station_id, start) -> pd.DataFrame:
    source = Hourly(station_id, start=pd.to_datetime(start), end=datetime(2025, 8, 31, 23, 59, 59))
    return source.fetch().reset_index()

def get_daily_data(station_id, start) -> pd.DataFrame:
    source = Daily(station_id, start=pd.to_datetime(start), end=datetime(2025, 8, 31, 23, 59, 59))
    return source.fetch().reset_index()

Denní data pro pět českých stanic do jednoho excel souboru:

In [ ]:
with pd.ExcelWriter("data-daily.xlsx") as writer:
    for station_id, station in selected_stations.iterrows():
        df = get_daily_data(station_id, station["daily_start"])
        df.to_excel(writer, sheet_name=station["name"].replace(" / ", "-"), index=False)  # název listu max 31 znaků

Data po hodině z Ruzyně:

In [ ]:
prague_hourly = get_hourly_data(11518, "1929-08-02")
prague_hourly.to_parquet("./prague-meteostat.parquet")
prague_hourly